In [ ]:
#Read data from Databricks#
df=spark.read.format(file_type)\ #csv,txt,json...
    .option('multiline','true')\ #is there multiple rows or not
    .option('inferSchema','true')\ #whether machine needs to automatically detect the schema of data or not
    .option('header','true')\ 
    .load(file_location) #location

In [ ]:
#Read data from DBFS to dataframe#
df.write.format(file_type)\
    .option('header','true')\
    .option('delimiter',',')\ #store the file, for each row we have several features and separated by comma
    .save(file_name)

In [ ]:
#simple manipulation in dataframe
df.cache()
display(df)
df.printSchema()
df.columns #get the name of each column
df.describe().show() #statistic summary of numeric column
type(object)

In [ ]:
df[[“column1”...”columnN”]] #create a new df with N columns, we write two bricks
df.withColumnRenamed(“old column name”, “new column name”)
df.withColumn(“column name”, df.[“column name”].cast(“data type”)) #create a new df assigning the data type for the column
df.withColumn(“new column name”, lit(default value)) #create a new df adding new columns with a default value
df.drop(“column name”) 
df.createOrReplaceTempView(“SQL temporary view name”)

In [ ]:
#Simple SQL Manipulations#
%sql 
select columnname from tablename where conditions
--column name does not include a space, we use _ to connect them

In [ ]:
#RDD#
#create RDD from external dataset file
sc = SparkContext(“local”, “texfile”)
rdd_new = sc.textFile(“/mnt/S3data/sample.text”)
#create RDD from existing dataframe
rdd_new = df.rdd
#convert RDD to dataframe
rdd_new.toDF()

#python function#
#1.
def fun_name(input):
    return output
#2.
lambda input: output

In [ ]:
#Transform RDD:
#Filter RDD: select some data in RDD to create a new RDD,similar to select in SQL
rdd.filter(lambda x: x[0]==`John’)
rdd.filter(lambda x: x[1][0]<=20) 
#Map:apply a function to each data in RDD
rdd.map(lambda x: (x[0], x[1][1]*0.393))
#union,join

#Actions in RDD:
collect
take(n)
count
reduce

In [ ]:
#Pandas Dataframe#
import pandas as pd
pd_new = df.toPandas()
pd.head(2)
df.tail(2)
pd.columns
pd.describe()
pd[[‘column1’, ‘column2’]]
pd[0:3] #select rows
pd.loc[0,’column’]
pd.iloc[0,1]
pd[‘colum1’,’colum2’].apply(fun)
pd.merge(pd1, pd2, on=‘column’)
pd.concat([pd1,pd2])
pd.dropna(axis=0, inplace=True)
pd.drop_duplicates(subset=‘column’, inplace=True)

In [ ]:
#Numpy#
np.array([[1,2,3],[4,5,6]])
np1 = pd.to_numpy() #conversion from pandas dataframe
np.zeros((2,2))
np.ones((2,2))
np.arange(0,3,0.1)


In [ ]:
#linear Regression with MLib#
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
From pyspark.ml.linalg import Vectors
# add a new column (feature column) to the original dataframe 
assembler = VectorAssembler (inputCols=[colum1, colum2, ...], outputCol=“features”)
new_df = assembler.transform(old_df)

In [ ]:
#model training#
train_dt, test_dt = df.randomSplit([0.7,0.3])
#run linear regression to estimate the parameters
lr = LinearRegression(labelCol=labelcolumname, featuresCol=featurecolumname)
#fit the model
lrmodel = lr.fit(train_dt)
#check the results:
lrmodel.intercept
lrmodel.coefficients

In [ ]:
#Model Testing#
test_results = lrmodel.evaluate(test_dt)
#check the testing result:
test_results.residuals
test_results.summary.meanSquaredError
test_results.rootMeanSquaredError

In [ ]:
#General Linear Model#
train_dt, test_dt = df.randomSplit([0.7,0.3])
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family=distribution type, link=link function type)
#for example:
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)

glrmodel = glr.fit(train_dt)
#training result using training dataset
glrmodel.intercept
glrmodel.coefficients
glrmodel.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

#evaluate the model on test dataset
test_results = model.evaluate(test_dt)
type(test_results)
test_results.residuals().show()
test_results.residuals().columns
test_results.residuals().createOrReplaceGlobalTempView("test")

In [ ]:
#Decision tree method#
from pyspark.ml.classification import DecisionTreeClassifier,RandomForestClassifier
#run decision tree models to estimate the parameters
dtc = DecisionTreeClassifier(labelCol=‘column’,featuresCol='column vector')
rfc = RandomForestClassifier(labelCol=‘column',featuresCol='column vector')
dtc_model = dtc.fit(train_dt)
rfc_model = rfc.fit(train_dt)
#test our model on test dataset, model comparison
dtc_model.transform(test_df)
rfc_model.transform(test_df)

#evaluation metrices
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="Private", predictionCol="prediction", metricName="accuracy")
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
#print the results
print("Here are the results!")
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))

In [ ]:
#XGBoost#
train, test = df.randomSplit([0.7, 0.3], seed = 0)
from sparkdl.xgboost import XgboostRegressor
xgb_regressor = XgboostRegressor(num_workers=3, labelCol="cnt", missing=0.0)
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
# Define a grid of hyperparameters to test:
#  - maxDepth: maximum depth of each decision tree 
#  - maxIter: iterations, or the total number of trees 
paramGrid = ParamGridBuilder()\
  .addGrid(xgb_regressor.max_depth, [2, 5])\
  .addGrid(xgb_regressor.n_estimators, [10, 100])\
  .build()
# Define an evaluation metric.  The CrossValidator compares the true labels with predicted values for each combination of parameters, and calculates this value to determine the best model.
evaluator = RegressionEvaluator(metricName="rmse",
                                labelCol=xgb_regressor.getLabelCol(),
                                predictionCol=xgb_regressor.getPredictionCol())
# Declare the CrossValidator, which performs the model tuning.
cv = CrossValidator(estimator=xgb_regressor, evaluator=evaluator, estimatorParamMaps=paramGrid)

In [ ]:
#FNN#
from pyspark.ml.classification import MultilayerPerceptronClassifier
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]
layers = [#input, #hidden1,..., #hidden K, #output]
# create the trainer and set its parameters,maxIter is the gradient desent parameter.
fnn = MultilayerPerceptronClassifier(maxIter=100, layers=layers, seed=1234)
# train the model
fnn_model = fnn.fit(train_df)
fnn_model.transform(test_df)
# compute accuracy on the test set
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
#Introduction to Keras#
#Build a softmax classifier with one layer#
from keras import models
from keras import layers
model=model.Sequential()
model.add(layers.Dense(10,activation='softmax',imput_shape=(784,)))
model.summary()

#train classifer#speficy optimzer, learning rate, loss function, metric
from keras import optimizers
model.compile(optimizers.RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#specify batch size & # of epoch：One pass over the n training samples.
history=model.fit(x_train_vec,y_train_vec,
                 batch_size=128,epochs=50,
                 validation_data=(x_valid_vec,y_valid_vec))
#evaluation results
loss_and_acc=model.evaluate(x_test_vec,y_test_vec)
print('loss=' +str(loss_and_acc[0]))
print('accuracy=' +str(loss_and_acc[1]))


#Build Full connected Netrual Network with 2-layers#
from keras import models 
from keras import layers 
d1 = 500 # width of the 1st hidden layer d1 = 500 
d2 = 500 # width of the 2nd hidden layer d2 = 500
model = models.Sequential() 
model.add(layers.Dense(d1, activation='relu', input_shape=(784,)))
model.add(layers.Dense(d2, activation='relu')) 
model.add(layers.Dense(10,activation='softmax'))
model.summary()
#train classifer
from keras import optimizers
model.compile(optimizers.RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#fit the model
history=model.fit(x_train_vec, y_train_vec,
                  batch_size=128,epochs=50,
                  validation_data=(x_valid_vec,y_valid_vec))
#evaluation results
loss_and_acc=model.evaluate(x_test_vec,y_test_vec)
print('loss=' +str(loss_and_acc[0]))
print('accuracy=' +str(loss_and_acc[1]))